In [27]:
import os
import json
import sqlite3
from datetime import datetime

# Connect to SQLite database (creates it if it doesn't exist)
db_path = "ipl_data.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create tables
cursor.executescript('''
    CREATE TABLE IF NOT EXISTS matches (
        match_id TEXT PRIMARY KEY,
        date TEXT,
        team1 TEXT,
        team2 TEXT,
        venue TEXT,
        toss_winner TEXT,
        toss_decision TEXT,
        winner TEXT,
        win_by TEXT,
        win_margin INTEGER,
        player_of_match TEXT
    );

    CREATE TABLE IF NOT EXISTS players (
        player_id TEXT,
        name TEXT,
        team TEXT,
        match_id TEXT,
        FOREIGN KEY (match_id) REFERENCES matches(match_id)
    );

    CREATE TABLE IF NOT EXISTS ball_by_ball (
        match_id TEXT,
        inning INTEGER,
        over INTEGER,
        ball INTEGER,
        batter TEXT,
        bowler TEXT,
        non_striker TEXT,
        runs_batter INTEGER,
        runs_extras INTEGER,
        runs_total INTEGER,
        wicket INTEGER,
        wicket_player_out TEXT,
        wicket_kind TEXT,
        FOREIGN KEY (match_id) REFERENCES matches(match_id)
    );
''')

# Folder containing JSON files
json_folder = "ipl_json"  # Replace with your folder path

# Process each JSON file
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(json_folder, filename)
        
        # Generate a unique match_id (e.g., from filename or date/team combo)
        match_id = filename.replace(".json", "")  # Simplistic; adjust as needed
        
        # Load JSON data
        with open(file_path, 'r') as f:
            data = json.load(f)

        # Extract match info
        info = data["info"]
        date = info["dates"][0]
        team1, team2 = info["teams"]
        venue = info["venue"]
        toss_winner = info["toss"]["winner"]
        toss_decision = info["toss"]["decision"]
        winner = info["outcome"].get("winner", "no result")
        win_by = list(info["outcome"].get("by", {}).keys())[0] if "by" in info["outcome"] else None
        win_margin = list(info["outcome"].get("by", {}).values())[0] if "by" in info["outcome"] else None
        player_of_match = info["player_of_match"][0] if info.get("player_of_match") else None

        # Insert match data
        cursor.execute('''
            INSERT OR IGNORE INTO matches (match_id, date, team1, team2, venue, toss_winner, toss_decision, winner, win_by, win_margin, player_of_match)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (match_id, date, team1, team2, venue, toss_winner, toss_decision, winner, win_by, win_margin, player_of_match))

        # Extract and insert player data
        for team, players in info["players"].items():
            for player in players:
                player_id = info["registry"]["people"].get(player, "unknown")
                cursor.execute('''
                    INSERT INTO players (player_id, name, team, match_id)
                    VALUES (?, ?, ?, ?)
                ''', (player_id, player, team, match_id))

        # Extract and insert ball-by-ball data
        for inning_idx, inning in enumerate(data["innings"], 1):
            team = inning["team"]
            for over in inning["overs"]:
                over_num = over["over"]
                for ball_idx, delivery in enumerate(over["deliveries"], 1):
                    batter = delivery["batter"]
                    bowler = delivery["bowler"]
                    non_striker = delivery["non_striker"]
                    runs_batter = delivery["runs"]["batter"]
                    runs_extras = delivery["runs"]["extras"]
                    runs_total = delivery["runs"]["total"]
                    
                    wicket = 1 if "wickets" in delivery else 0
                    wicket_player_out = delivery["wickets"][0]["player_out"] if wicket else None
                    wicket_kind = delivery["wickets"][0]["kind"] if wicket else None

                    cursor.execute('''
                        INSERT INTO ball_by_ball (match_id, inning, over, ball, batter, bowler, non_striker, runs_batter, runs_extras, runs_total, wicket, wicket_player_out, wicket_kind)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (match_id, inning_idx, over_num, ball_idx, batter, bowler, non_striker, runs_batter, runs_extras, runs_total, wicket, wicket_player_out, wicket_kind))

# Commit changes and close connection
conn.commit()
conn.close()

print(f"Database created successfully at {db_path}")

Database created successfully at ipl_data.db


In [28]:
conn = sqlite3.connect("ipl_data.db")
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM matches")
print(f"Total matches: {cursor.fetchone()[0]}")
conn.close()

Total matches: 1100


In [29]:
import sqlite3

# Connect to the SQLite database
db_path = "ipl_data.db"  # Replace with your database path if different
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query to extract unique teams from matches and players tables
cursor.execute('''
    SELECT DISTINCT team_name FROM (
        SELECT team1 AS team_name FROM matches
        UNION
        SELECT team2 AS team_name FROM matches
        UNION
        SELECT team AS team_name FROM players
    ) AS all_teams
    WHERE team_name IS NOT NULL
    ORDER BY team_name
''')

# Fetch and print the unique teams
unique_teams = cursor.fetchall()
print("Unique Teams:")
for team in unique_teams:
    print(team[0])  # team[0] because fetchall() returns a list of tuples

# Print the total count
print(f"\nTotal number of unique teams: {len(unique_teams)}")

# Close the connection
conn.close()

Unique Teams:
Chennai Super Kings
Deccan Chargers
Delhi Capitals
Delhi Daredevils
Gujarat Lions
Gujarat Titans
Kings XI Punjab
Kochi Tuskers Kerala
Kolkata Knight Riders
Lucknow Super Giants
Mumbai Indians
Pune Warriors
Punjab Kings
Rajasthan Royals
Rising Pune Supergiant
Rising Pune Supergiants
Royal Challengers Bangalore
Royal Challengers Bengaluru
Sunrisers Hyderabad

Total number of unique teams: 19


In [ ]:
import sqlite3

# Define the team name mapping
team_mapping = {
    "Deccan Chargers": "Sunrisers Hyderabad",
    "Delhi Daredevils": "Delhi Capitals",
    "Kings XI Punjab": "Punjab Kings",
    "Rising Pune Supergiants": "Rising Pune Supergiant",
    "Royal Challengers Bangalore": "Royal Challengers Bengaluru",
    # Teams with no changes or defunct teams remain as-is
    "Chennai Super Kings": "Chennai Super Kings",
    "Delhi Capitals": "Delhi Capitals",
    "Gujarat Lions": "Gujarat Lions",
    "Gujarat Titans": "Gujarat Titans",
    "Kochi Tuskers Kerala": "Kochi Tuskers Kerala",
    "Kolkata Knight Riders": "Kolkata Knight Riders",
    "Lucknow Super Giants": "Lucknow Super Giants",
    "Mumbai Indians": "Mumbai Indians",
    "Pune Warriors": "Pune Warriors",
    "Punjab Kings": "Punjab Kings",
    "Rajasthan Royals": "Rajasthan Royals",
    "Rising Pune Supergiant": "Rising Pune Supergiant",
    "Royal Challengers Bengaluru": "Royal Challengers Bengaluru",
    "Sunrisers Hyderabad": "Sunrisers Hyderabad"
}

# Connect to the SQLite database
db_path = "ipl_data.db"  # Replace with your database path if different
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Update team names in the matches table
for old_name, new_name in team_mapping.items():
    if old_name != new_name:  
        cursor.execute('''
            UPDATE matches 
            SET team1 = ? 
            WHERE team1 = ?
        ''', (new_name, old_name))
        
        cursor.execute('''
            UPDATE matches 
            SET team2 = ? 
            WHERE team2 = ?
        ''', (new_name, old_name))
        
        cursor.execute('''
            UPDATE matches 
            SET toss_winner = ? 
            WHERE toss_winner = ?
        ''', (new_name, old_name))
        
        cursor.execute('''
            UPDATE matches 
            SET winner = ? 
            WHERE winner = ?
        ''', (new_name, old_name))

# Update team names in the players table
for old_name, new_name in team_mapping.items():
    if old_name != new_name:
        cursor.execute('''
            UPDATE players 
            SET team = ? 
            WHERE team = ?
        ''', (new_name, old_name))

# Commit changes
conn.commit()

# Verify the updated unique teams
cursor.execute('''
    SELECT DISTINCT team_name FROM (
        SELECT team1 AS team_name FROM matches
        UNION
        SELECT team2 AS team_name FROM matches
        UNION
        SELECT team AS team_name FROM players
    ) AS all_teams
    WHERE team_name IS NOT NULL
    ORDER BY team_name
''')

updated_teams = cursor.fetchall()
print("Updated Unique Teams:")
for team in updated_teams:
    print(team[0])
print(f"\nTotal number of unique teams: {len(updated_teams)}")

# Close the connection
conn.close()

Updated Unique Teams:
Chennai Super Kings
Delhi Capitals
Gujarat Lions
Gujarat Titans
Kochi Tuskers Kerala
Kolkata Knight Riders
Lucknow Super Giants
Mumbai Indians
Pune Warriors
Punjab Kings
Rajasthan Royals
Rising Pune Supergiant
Royal Challengers Bengaluru
Sunrisers Hyderabad

Total number of unique teams: 14


In [31]:
import sqlite3
from datetime import datetime

# Connect to the database
def connect_db(db_path="ipl_data.db"):
    return sqlite3.connect(db_path)

# Feature extraction function based on team names only
def extract_features(team1, team2, latest_date="2025-03-26"):
    conn = connect_db()
    cursor = conn.cursor()

    features = {
        "team1": team1,
        "team2": team2,
        "date": latest_date,  # Use the latest date as a cutoff for historical data
    }

    # 1. Team-Level Features
    for team in [team1, team2]:
        # Historical Win Rate
        cursor.execute("""
            SELECT COUNT(CASE WHEN winner = ? THEN 1 END) * 100.0 / COUNT(*) 
            FROM matches 
            WHERE (team1 = ? OR team2 = ?) AND date <= ?
        """, (team, team, team, latest_date))
        features[f"{team}_win_rate"] = cursor.fetchone()[0] or 0.0

        # Head-to-Head Win Rate (team1's perspective)
        cursor.execute("""
            SELECT COUNT(CASE WHEN winner = ? THEN 1 END) * 100.0 / COUNT(*) 
            FROM matches 
            WHERE ((team1 = ? AND team2 = ?) OR (team1 = ? AND team2 = ?)) AND date <= ?
        """, (team1, team1, team2, team2, team1, latest_date))
        features[f"{team1}_vs_{team2}_win_rate"] = cursor.fetchone()[0] or 0.0

        # Recent Form (Last 5 Matches)
        cursor.execute("""
            SELECT winner 
            FROM matches 
            WHERE (team1 = ? OR team2 = ?) AND date <= ? 
            ORDER BY date DESC 
            LIMIT 5
        """, (team, team, latest_date))
        recent_matches = cursor.fetchall()
        wins = sum(1 for match in recent_matches if match[0] == team)
        features[f"{team}_recent_form"] = (wins / 5.0) * 100 if recent_matches else 0.0

        # Average Runs Scored (across all matches)
        cursor.execute("""
            SELECT AVG(runs_total) 
            FROM (
                SELECT match_id, inning, SUM(runs_total) as runs_total 
                FROM ball_by_ball 
                WHERE match_id IN (
                    SELECT match_id FROM matches 
                    WHERE (team1 = ? OR team2 = ?) AND date <= ?
                )
                GROUP BY match_id, inning
            ) WHERE runs_total > 0
        """, (team, team, latest_date))
        features[f"{team}_avg_runs"] = cursor.fetchone()[0] or 0.0

        # Average Wickets Taken (across all matches)
        cursor.execute("""
            SELECT AVG(wickets) 
            FROM (
                SELECT match_id, inning, COUNT(wicket) as wickets 
                FROM ball_by_ball 
                WHERE match_id IN (
                    SELECT match_id FROM matches 
                    WHERE (team1 = ? OR team2 = ?) AND date <= ?
                )
                GROUP BY match_id, inning
            ) WHERE wickets > 0
        """, (team, team, latest_date))
        features[f"{team}_wickets_taken"] = cursor.fetchone()[0] or 0.0

        # Venue Win Rate (averaged across all venues)
        cursor.execute("""
            SELECT AVG(venue_win_rate) 
            FROM (
                SELECT venue, COUNT(CASE WHEN winner = ? THEN 1 END) * 100.0 / COUNT(*) as venue_win_rate 
                FROM matches 
                WHERE (team1 = ? OR team2 = ?) AND date <= ?
                GROUP BY venue
            )
        """, (team, team, team, latest_date))
        features[f"{team}_venue_win_rate"] = cursor.fetchone()[0] or 0.0

    # 2. Player-Level Features (Top performers based on all data)
    for team in [team1, team2]:
        # Get all players who have played for the team
        cursor.execute("""
            SELECT DISTINCT name 
            FROM players 
            WHERE team = ? AND match_id IN (
                SELECT match_id FROM matches WHERE date <= ?
            )
        """, (team, latest_date))
        players = [row[0] for row in cursor.fetchall()]

        # Top 3 Batsmen by Total Runs
        batsmen_stats = []
        for player in players:
            cursor.execute("""
                SELECT SUM(runs_batter) 
                FROM ball_by_ball 
                WHERE batter = ? AND match_id IN (
                    SELECT match_id FROM matches 
                    WHERE (team1 = ? OR team2 = ?) AND date <= ?
                )
            """, (player, team, team, latest_date))
            runs = cursor.fetchone()[0] or 0
            batsmen_stats.append((player, runs))
        top_batsmen = sorted(batsmen_stats, key=lambda x: x[1], reverse=True)[:3]

        for i, (batsman, _) in enumerate(top_batsmen):
            cursor.execute("""
                SELECT SUM(runs_batter) * 100.0 / COUNT(*) 
                FROM ball_by_ball 
                WHERE batter = ? AND match_id IN (
                    SELECT match_id FROM matches WHERE date <= ?
                )
            """, (batsman, latest_date))
            strike_rate = cursor.fetchone()[0] or 0.0
            features[f"{team}_batsman_{i+1}_name"] = batsman
            features[f"{team}_batsman_{i+1}_strike_rate"] = strike_rate

        # Top 3 Bowlers by Total Wickets
        bowlers_stats = []
        for player in players:
            cursor.execute("""
                SELECT COUNT(wicket) 
                FROM ball_by_ball 
                WHERE bowler = ? AND match_id IN (
                    SELECT match_id FROM matches 
                    WHERE (team1 = ? OR team2 = ?) AND date <= ?
                )
            """, (player, team, team, latest_date))
            wickets = cursor.fetchone()[0] or 0
            bowlers_stats.append((player, wickets))
        top_bowlers = sorted(bowlers_stats, key=lambda x: x[1], reverse=True)[:3]

        for i, (bowler, _) in enumerate(top_bowlers):
            cursor.execute("""
                SELECT SUM(runs_total) * 6.0 / COUNT(*) 
                FROM ball_by_ball 
                WHERE bowler = ? AND match_id IN (
                    SELECT match_id FROM matches WHERE date <= ?
                )
            """, (bowler, latest_date))
            economy_rate = cursor.fetchone()[0] or 0.0
            features[f"{team}_bowler_{i+1}_name"] = bowler
            features[f"{team}_bowler_{i+1}_economy"] = economy_rate

    # 3. Contextual Features
    for team in [team1, team2]:
        # Chasing Success Rate
        cursor.execute("""
            SELECT COUNT(CASE WHEN winner = toss_winner AND toss_decision = 'field' THEN 1 
                              WHEN winner != toss_winner AND toss_decision = 'bat' THEN 1 END) * 100.0 / COUNT(*) 
            FROM matches 
            WHERE (team1 = ? OR team2 = ?) AND date <= ?
        """, (team, team, latest_date))
        features[f"{team}_chase_win_rate"] = cursor.fetchone()[0] or 0.0

        # Average Powerplay Runs
        cursor.execute("""
            SELECT AVG(powerplay_runs) 
            FROM (
                SELECT match_id, inning, SUM(runs_total) as powerplay_runs 
                FROM ball_by_ball 
                WHERE over < 6 AND match_id IN (
                    SELECT match_id FROM matches 
                    WHERE (team1 = ? OR team2 = ?) AND date <= ?
                )
                GROUP BY match_id, inning
            )
        """, (team, team, latest_date))
        features[f"{team}_powerplay_runs"] = cursor.fetchone()[0] or 0.0

        # Average Death Over Runs
        cursor.execute("""
            SELECT AVG(death_runs) 
            FROM (
                SELECT match_id, inning, SUM(runs_total) as death_runs 
                FROM ball_by_ball 
                WHERE over >= 15 AND match_id IN (
                    SELECT match_id FROM matches 
                    WHERE (team1 = ? OR team2 = ?) AND date <= ?
                )
                GROUP BY match_id, inning
            )
        """, (team, team, latest_date))
        features[f"{team}_death_runs"] = cursor.fetchone()[0] or 0.0

    conn.close()
    return features

# Main execution with user input
if __name__ == "__main__":
    # Get team names from user
    team1 = input("Enter the name of Team 1: ").strip()
    team2 = input("Enter the name of Team 2: ").strip()

    try:
        # Extract features
        features = extract_features(team1, team2)
        
        # Print extracted features
        print(f"\nExtracted Features for {team1} vs {team2}:")
        for key, value in features.items():
            print(f"{key}: {value}")

    except Exception as e:
        print(f"Error: {e}")


Extracted Features for Chennai Super Kings vs Mumbai Indians:
team1: Chennai Super Kings
team2: Mumbai Indians
date: 2025-03-26
Chennai Super Kings_win_rate: 58.15899581589958
Chennai Super Kings_vs_Mumbai Indians_win_rate: 47.36842105263158
Chennai Super Kings_recent_form: 60.0
Chennai Super Kings_avg_runs: 317.9498956158664
Chennai Super Kings_wickets_taken: 239.9582463465553
Chennai Super Kings_venue_win_rate: 49.29671563660925
Mumbai Indians_win_rate: 54.19847328244275
Mumbai Indians_recent_form: 20.0
Mumbai Indians_avg_runs: 313.80149812734084
Mumbai Indians_wickets_taken: 236.64794007490636
Mumbai Indians_venue_win_rate: 46.43063742363163
Chennai Super Kings_batsman_1_name: SK Raina
Chennai Super Kings_batsman_1_strike_rate: 132.53531242518554
Chennai Super Kings_batsman_2_name: MS Dhoni
Chennai Super Kings_batsman_2_strike_rate: 132.7677893137503
Chennai Super Kings_batsman_3_name: F du Plessis
Chennai Super Kings_batsman_3_strike_rate: 133.14037626628075
Chennai Super Kings_bo

In [ ]:
import os
import sqlite3
import openai
from datetime import datetime

# =============================================================================
# 1. OpenAI Client Setup
# =============================================================================
# Ensure you have set OPENAI_API_KEY as an environment variable
# or replace os.getenv("OPENAI_API_KEY") with your actual key.
OPENAI_API_KEY=""


# =============================================================================
# 2. Team Roster (IPL 2025)
# =============================================================================
ipl_teams_2025 = {
    "Chennai Super Kings": {
        "players": [
            "Ruturaj Gaikwad",
            "Devon Conway",
            "Moeen Ali",
            "Ravindra Jadeja",
            "MS Dhoni",
            "Shivam Dube",
            "Deepak Chahar",
            "Matheesha Pathirana",
            "Tushar Deshpande",
            "Ajinkya Rahane",
            "Mitchell Santner"
        ]
    },
    "Mumbai Indians": {
        "players": [
            "Rohit Sharma",
            "Ishan Kishan",
            "Suryakumar Yadav",
            "Tilak Varma",
            "Hardik Pandya",
            "Tim David",
            "Jasprit Bumrah",
            "Akash Madhwal",
            "Piyush Chawla",
            "Gerald Coetzee",
            "Shams Mulani"
        ]
    },
    "Royal Challengers Bangalore": {
        "players": [
            "Virat Kohli",
            "Faf du Plessis",
            "Glenn Maxwell",
            "Rajat Patidar",
            "Cameron Green",
            "Dinesh Karthik",
            "Mahipal Lomror",
            "Mohammed Siraj",
            "Karn Sharma",
            "Alzarri Joseph",
            "Akash Deep"
        ]
    },
    "Kolkata Knight Riders": {
        "players": [
            "Shreyas Iyer",
            "Nitish Rana",
            "Rahmanullah Gurbaz",
            "Venkatesh Iyer",
            "Andre Russell",
            "Rinku Singh",
            "Sunil Narine",
            "Mitchell Starc",
            "Varun Chakaravarthy",
            "Harshit Rana",
            "Suyash Sharma"
        ]
    },
    "Delhi Capitals": {
        "players": [
            "David Warner",
            "Prithvi Shaw",
            "Mitchell Marsh",
            "Rishabh Pant",
            "Axar Patel",
            "Lalit Yadav",
            "Kuldeep Yadav",
            "Anrich Nortje",
            "Khaleel Ahmed",
            "Mukesh Kumar",
            "Tristan Stubbs"
        ]
    },
    "Rajasthan Royals": {
        "players": [
            "Yashasvi Jaiswal",
            "Jos Buttler",
            "Sanju Samson",
            "Shimron Hetmyer",
            "Dhruv Jurel",
            "Riyan Parag",
            "Ravichandran Ashwin",
            "Trent Boult",
            "Yuzvendra Chahal",
            "Avesh Khan",
            "Adam Zampa"
        ]
    },
    "Punjab Kings": {
        "players": [
            "Shikhar Dhawan",
            "Jonny Bairstow",
            "Prabhsimran Singh",
            "Liam Livingstone",
            "Jitesh Sharma",
            "Sam Curran",
            "Sikandar Raza",
            "Harpreet Brar",
            "Kagiso Rabada",
            "Arshdeep Singh",
            "Harshal Patel"
        ]
    },
    "Sunrisers Hyderabad": {
        "players": [
            "Travis Head",
            "Abhishek Sharma",
            "Aiden Markram",
            "Heinrich Klaasen",
            "Abdul Samad",
            "Shahbaz Ahmed",
            "Pat Cummins",
            "Washington Sundar",
            "Bhuvneshwar Kumar",
            "T Natarajan",
            "Mayank Markande"
        ]
    },
    "Lucknow Super Giants": {
        "players": [
            "KL Rahul",
            "Quinton de Kock",
            "Devdutt Padikkal",
            "Marcus Stoinis",
            "Nicholas Pooran",
            "Ayush Badoni",
            "Krunal Pandya",
            "Ravi Bishnoi",
            "Naveen-ul-Haq",
            "Mohsin Khan",
            "Mayank Yadav"
        ]
    },
    "Gujarat Titans": {
        "players": [
            "Shubman Gill",
            "Wriddhiman Saha",
            "Sai Sudharsan",
            "David Miller",
            "Vijay Shankar",
            "Rahul Tewatia",
            "Rashid Khan",
            "Noor Ahmad",
            "Mohit Sharma",
            "Umesh Yadav",
            "Spencer Johnson"
        ]
    }
}

# =============================================================================
# 3. Database Utility
# =============================================================================
def connect_db(db_path="ipl_data.db"):
    """Connect to the local SQLite DB file."""
    return sqlite3.connect(db_path)

# =============================================================================
# 4. Feature Extraction
# =============================================================================
def extract_features(team1, team2, latest_date="2025-03-26"):
    """
    Extract features for the two teams, including:
      - Team-level features (win rate, recent form, etc.)
      - Player-level features (runs, strike rate, wickets, economy, etc.)
    Only includes players from the ipl_teams_2025 dictionary.
    """
    conn = connect_db()
    cursor = conn.cursor()

    features = {
        "team1": team1,
        "team2": team2,
        "date": latest_date,
    }

    # ---------- Team-Level Features ----------
    for team in [team1, team2]:
        cursor.execute("""
            SELECT COUNT(CASE WHEN winner = ? THEN 1 END) * 100.0 / COUNT(*)
            FROM matches
            WHERE (team1 = ? OR team2 = ?) AND date <= ?
        """, (team, team, team, latest_date))
        features[f"{team}_win_rate"] = cursor.fetchone()[0] or 0.0

        cursor.execute("""
            SELECT COUNT(CASE WHEN winner = ? THEN 1 END) * 100.0 / COUNT(*)
            FROM matches
            WHERE ((team1 = ? AND team2 = ?) OR (team1 = ? AND team2 = ?))
              AND date <= ?
        """, (team1, team1, team2, team2, team1, latest_date))
        features[f"{team1}_vs_{team2}_win_rate"] = cursor.fetchone()[0] or 0.0

        # recent_form (last 5 matches)
        cursor.execute("""
            SELECT winner
            FROM matches
            WHERE (team1 = ? OR team2 = ?) AND date <= ?
            ORDER BY date DESC
            LIMIT 5
        """, (team, team, latest_date))
        recent_matches = cursor.fetchall()
        wins = sum(1 for match in recent_matches if match[0] == team)
        features[f"{team}_recent_form"] = (wins / 5.0) * 100 if recent_matches else 0.0

        # average runs
        cursor.execute("""
            SELECT AVG(runs_total)
            FROM (
                SELECT match_id, inning, SUM(runs_total) AS runs_total
                FROM ball_by_ball
                WHERE match_id IN (
                    SELECT match_id FROM matches
                    WHERE (team1 = ? OR team2 = ?) AND date <= ?
                )
                GROUP BY match_id, inning
            ) WHERE runs_total > 0
        """, (team, team, latest_date))
        features[f"{team}_avg_runs"] = cursor.fetchone()[0] or 0.0

        # average wickets
        cursor.execute("""
            SELECT AVG(wickets)
            FROM (
                SELECT match_id, inning, COUNT(wicket) AS wickets
                FROM ball_by_ball
                WHERE match_id IN (
                    SELECT match_id FROM matches
                    WHERE (team1 = ? OR team2 = ?) AND date <= ?
                )
                GROUP BY match_id, inning
            ) WHERE wickets > 0
        """, (team, team, latest_date))
        features[f"{team}_wickets_taken"] = cursor.fetchone()[0] or 0.0

        # average venue win rate
        cursor.execute("""
            SELECT AVG(venue_win_rate)
            FROM (
                SELECT venue,
                       COUNT(CASE WHEN winner = ? THEN 1 END) * 100.0 / COUNT(*) AS venue_win_rate
                FROM matches
                WHERE (team1 = ? OR team2 = ?)
                  AND date <= ?
                GROUP BY venue
            )
        """, (team, team, team, latest_date))
        features[f"{team}_venue_win_rate"] = cursor.fetchone()[0] or 0.0

    # ---------- Player-Level Features (Only current roster) ----------
    for team in [team1, team2]:
        players = ipl_teams_2025.get(team, {}).get("players", [])

        # Batting features
        for player in players:
            cursor.execute("""
                SELECT SUM(runs_batter)
                FROM ball_by_ball
                WHERE batter = ?
                  AND match_id IN (
                      SELECT match_id FROM matches
                      WHERE (team1 = ? OR team2 = ?)
                        AND date <= ?
                  )
            """, (player, team, team, latest_date))
            total_runs = cursor.fetchone()[0] or 0

            cursor.execute("""
                SELECT SUM(runs_batter) * 100.0 / COUNT(*)
                FROM ball_by_ball
                WHERE batter = ?
                  AND match_id IN (
                      SELECT match_id FROM matches
                      WHERE date <= ?
                  )
            """, (player, latest_date))
            strike_rate = cursor.fetchone()[0] or 0.0

            if total_runs > 0:
                features[f"{team}_batsman_{player}_total_runs"] = total_runs
                features[f"{team}_batsman_{player}_strike_rate"] = strike_rate

        # Bowling features
        for player in players:
            cursor.execute("""
                SELECT COUNT(wicket)
                FROM ball_by_ball
                WHERE bowler = ?
                  AND match_id IN (
                      SELECT match_id FROM matches
                      WHERE (team1 = ? OR team2 = ?)
                        AND date <= ?
                  )
            """, (player, team, team, latest_date))
            total_wickets = cursor.fetchone()[0] or 0

            cursor.execute("""
                SELECT SUM(runs_total) * 6.0 / COUNT(*)
                FROM ball_by_ball
                WHERE bowler = ?
                  AND match_id IN (
                      SELECT match_id FROM matches
                      WHERE date <= ?
                  )
            """, (player, latest_date))
            economy_rate = cursor.fetchone()[0] or 0.0

            if total_wickets > 0:
                features[f"{team}_bowler_{player}_total_wickets"] = total_wickets
                features[f"{team}_bowler_{player}_economy"] = economy_rate

    # ---------- Contextual Features ----------
    for team in [team1, team2]:
        # chase_win_rate
        cursor.execute("""
            SELECT COUNT(
                CASE
                    WHEN winner = toss_winner AND toss_decision = 'field' THEN 1
                    WHEN winner != toss_winner AND toss_decision = 'bat' THEN 1
                END
            ) * 100.0 / COUNT(*)
            FROM matches
            WHERE (team1 = ? OR team2 = ?) AND date <= ?
        """, (team, team, latest_date))
        features[f"{team}_chase_win_rate"] = cursor.fetchone()[0] or 0.0

        # powerplay_runs
        cursor.execute("""
            SELECT AVG(powerplay_runs)
            FROM (
                SELECT match_id, inning, SUM(runs_total) AS powerplay_runs
                FROM ball_by_ball
                WHERE over < 6
                  AND match_id IN (
                      SELECT match_id FROM matches
                      WHERE (team1 = ? OR team2 = ?)
                        AND date <= ?
                  )
                GROUP BY match_id, inning
            )
        """, (team, team, latest_date))
        features[f"{team}_powerplay_runs"] = cursor.fetchone()[0] or 0.0

        # death_runs
        cursor.execute("""
            SELECT AVG(death_runs)
            FROM (
                SELECT match_id, inning, SUM(runs_total) AS death_runs
                FROM ball_by_ball
                WHERE over >= 15
                  AND match_id IN (
                      SELECT match_id FROM matches
                      WHERE (team1 = ? OR team2 = ?)
                        AND date <= ?
                  )
                GROUP BY match_id, inning
            )
        """, (team, team, latest_date))
        features[f"{team}_death_runs"] = cursor.fetchone()[0] or 0.0

    conn.close()
    return features

# =============================================================================
# 5. Prompt Formatting
# =============================================================================
def format_prompt(features):
    """
    Builds a thorough prompt for the LLM with all relevant stats.
    """
    prompt = f"Predict the outcome of the IPL match with the following data:\n\n"
    prompt += f"Match Date: {features['date']}\n"
    prompt += f"Teams: {features['team1']} vs {features['team2']}\n\n"

    prompt += "Team-Level Statistics:\n"
    for key, value in features.items():
        if key in ["team1", "team2", "date"]:
            continue
        if any(metric in key for metric in [
            "win_rate", "recent_form", "avg_runs", "wickets_taken",
            "venue_win_rate", "chase_win_rate", "powerplay_runs", "death_runs"
        ]):
            prompt += f"{key}: {value}\n"

    prompt += "\nPlayer-Level Statistics:\n"
    for key, value in features.items():
        if any(metric in key for metric in ["batsman", "bowler"]):
            prompt += f"{key}: {value}\n"

    # Instruction to LLM
    prompt += (
        "\nBased on the above detailed data, provide a match prediction. "
        "Explain your reasoning in detail by discussing team performance, "
        "Calcualte the percentage win for each team and Your final verdict at the end."
    )
    return prompt

# =============================================================================
# 6. LLM Prediction Function (Using openai>=1.0.0)
# =============================================================================
def get_prediction_from_llm(prompt: str) -> str:
    """
    Calls the new openai>=1.0.0 interface with 'gpt-4o-mini' via chat.completions.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a sports analyst specialized in cricket and IPL statistics."},
                {"role": "user", "content": prompt}
            ],
            #temperature=0.7,
            #max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error while calling OpenAI API:\n{e}"

# =============================================================================
# 7. Main Execution (Team Selection)
# =============================================================================
if __name__ == "__main__":

    teams = [
        "Chennai Super Kings", "Delhi Capitals", "Gujarat Lions", "Gujarat Titans",
        "Kochi Tuskers Kerala", "Kolkata Knight Riders", "Lucknow Super Giants",
        "Mumbai Indians", "Pune Warriors", "Punjab Kings", "Rajasthan Royals",
        "Rising Pune Supergiant", "Royal Challengers Bengaluru", "Sunrisers Hyderabad"
    ]

    print("Available Teams:")
    for i, t in enumerate(teams, 1):
        print(f"{i}. {t}")

    try:
        # User selects team 1
        team1 = teams[int(input("Select Team 1 (number): ")) - 1]
        # User selects team 2
        team2 = teams[int(input("Select Team 2 (number): ")) - 1]
        if team1 == team2:
            raise ValueError("Teams must be different.")

        # Extract features
        features = extract_features(team1, team2)
        # Format prompt
        prompt = format_prompt(features)
        print("\nPrompt sent to LLM:\n", prompt)

        # Get prediction from LLM
        prediction = get_prediction_from_llm(prompt)
        print("\nPrediction:\n", prediction)

    except Exception as e:
        print(f"Error: {e}")


Available Teams:
1. Chennai Super Kings
2. Delhi Capitals
3. Gujarat Lions
4. Gujarat Titans
5. Kochi Tuskers Kerala
6. Kolkata Knight Riders
7. Lucknow Super Giants
8. Mumbai Indians
9. Pune Warriors
10. Punjab Kings
11. Rajasthan Royals
12. Rising Pune Supergiant
13. Royal Challengers Bengaluru
14. Sunrisers Hyderabad

Prompt sent to LLM:
 Predict the outcome of the IPL match with the following data:

Match Date: 2025-03-26
Teams: Chennai Super Kings vs Royal Challengers Bengaluru

Team-Level Statistics:
Chennai Super Kings_win_rate: 58.15899581589958
Chennai Super Kings_vs_Royal Challengers Bengaluru_win_rate: 65.625
Chennai Super Kings_recent_form: 60.0
Chennai Super Kings_avg_runs: 317.9498956158664
Chennai Super Kings_wickets_taken: 239.9582463465553
Chennai Super Kings_venue_win_rate: 49.29671563660925
Royal Challengers Bengaluru_win_rate: 47.65625
Royal Challengers Bengaluru_recent_form: 80.0
Royal Challengers Bengaluru_avg_runs: 316.3404255319149
Royal Challengers Bengaluru_wi